# Document Analysis in Python
In this notebook we will cover:
- Reading document data into memory
- Creating bag of words features
- Creating smoothed tf-idf features

In [1]:
import requests
import json
from contextlib import closing

# get API key saved on hardrive
with open('../NYTimesAPI.txt') as f:
    api_key = f.read() # read in my private key (sorry, not in this repo ¯\_(ツ)_/¯ )
    
# make base URL and dictionary of get request key/values
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json"
payload = {'api-key': api_key, 'q':'world cup'} # key/values for get request (look up in api, there are lots)

# Perform the actual request
with closing(requests.get(url, params=payload)) as r:
    articles = r.json()
    print(articles)

{'status': 'OK', 'response': {'docs': [{'word_count': 0, 'multimedia': [], 'type_of_material': 'SectionFront', '_id': '53c00c2038f0d85f5c71ae10', 'headline': {'main': 'World Cup', 'print_headline': None, 'name': None, 'sub': None, 'seo': None, 'content_kicker': None, 'kicker': None}, 'score': 2.3329575, 'keywords': [], 'blog': {}, 'document_type': 'sectionfront', 'web_url': 'https://www.nytimes.com/pages/sports/worldcup/index.html', 'snippet': ''}, {'word_count': 548, 'uri': 'nyt://article/b9596925-8224-5c9a-b31c-d4f84d329c2a', 'type_of_material': 'News', 'snippet': 'The Americans defeated Belarus, 3-2, after CoCo Vandeweghe and Shelby Rogers won the deciding doubles match to claim their first title since 2000.', '_id': '5a08d3fe7c459f246b634065', 'blog': {}, 'document_type': 'article', 'source': 'The New York Times', 'web_url': 'https://www.nytimes.com/2017/11/12/sports/tennis/fed-cup-final.html', 'pub_date': '2017-11-12T23:06:16+0000', 'multimedia': [{'legacy': {'xlargeheight': 387, 

In [ ]:
# OR we can load an example query like this
# run this block of code if you can't run anything else
import json 
with open('data/nytime.json') as f:
    articles = json.loads(f.read())
    
articles

In [2]:
# get the summary text from each article
#  lead_paragraph  is no snippet
summary_text = [x['snippet'] for x in articles['response']['docs']]
summary_text

['',
 'The Americans defeated Belarus, 3-2, after CoCo Vandeweghe and Shelby Rogers won the deciding doubles match to claim their first title since 2000.',
 'Five months after declaring its interest in hosting the 2026 World Cup, Morocco hasn’t said much about how it proposes to stage the world’s most-watched sporting event.',
 '<p>News about the World Cup, including commentary and archival articles published in The New York Times.</p>',
 '<p>News about the World Cup, including commentary and archival articles published in The New York Times.</p>',
 'The 5-year-old is one of the most popular horses in Japan, and he will be retiring at the end of the year.',
 "News about Breeders' Cup World Thoroughbred Championships, including commentary and archival articles published in The New York Times.",
 'Soccer’s scandal-marred governing body pays nonexecutive board members more than some of the biggest companies in the world, a concern to governance experts.',
 'Shiffrin, best known for the mo

# Converting document data into different representations
First lets go through and count the unique words in each opening sentence (that is what the NYTimes give us for free).
- http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer() # an object capable of counting words in a document!

# count_vect.fit(summary_text)
# count_vect.transform(summary_text)
bag_words = count_vect.fit_transform(summary_text)

In [6]:
print(bag_words.shape) # this is a sparse matrix
print('=========')
print(bag_words[1])

(10, 118)
  (0, 1)	1
  (0, 91)	1
  (0, 105)	1
  (0, 41)	1
  (0, 102)	1
  (0, 21)	1
  (0, 106)	1
  (0, 66)	1
  (0, 34)	1
  (0, 28)	1
  (0, 114)	1
  (0, 86)	1
  (0, 89)	1
  (0, 6)	1
  (0, 107)	1
  (0, 22)	1
  (0, 4)	1
  (0, 11)	1
  (0, 30)	1
  (0, 5)	1
  (0, 101)	2


# Self Test: ML01b.3: 
Do you expect the vocabulary from the articles above to be:
- A. Greater than 1M words
- B. Greater than 10,000 words
- C. Fewer than 10,000 words

In [7]:
print(len(count_vect.vocabulary_))
print(count_vect.vocabulary_)

118
{'dispute': 32, 'declaring': 29, 'vandeweghe': 107, 'york': 117, 'horses': 50, 'thoroughbred': 103, 'issuing': 59, 'has': 47, 'how': 52, 'and': 6, 'event': 37, 'since': 91, 'hasn': 48, 'proposes': 82, 'title': 105, 'championships': 20, 'year': 116, 'in': 54, 'cup': 27, 'immediately': 53, 'the': 101, 'hosting': 51, 'best': 12, 'marred': 65, 'match': 66, 'interest': 56, 'published': 83, 'bested': 13, 'sporting': 95, 'body': 16, 'nonexecutive': 75, 'end': 35, 'claim': 21, 'pays': 80, 'biggest': 14, 'slalom': 92, 'soccer': 93, 'retiring': 85, 'giant': 44, 'members': 67, 'japan': 62, 'to': 106, 'companies': 24, 'with': 113, 'of': 76, 'known': 63, 'than': 100, 'new': 73, 'will': 112, 'five': 42, 'he': 49, 'at': 9, 'experts': 39, 'americans': 5, 'governing': 46, 'watched': 110, 'including': 55, 'rogers': 86, 'for': 43, 'coco': 22, 'articles': 8, 'some': 94, 'commentary': 23, 'morocco': 70, 'court': 26, 'patent': 79, 'most': 71, 'one': 78, 'belarus': 11, 'technical': 99, 'stop': 98, 'after

In [9]:
# we can still look at the data using an inverse transform
# but we lose the ordering of the words (after all its just a bag of wrods model)
count_vect.inverse_transform(bag_words[1])

[array(['2000', 'since', 'title', 'first', 'their', 'claim', 'to', 'match',
        'doubles', 'deciding', 'won', 'rogers', 'shelby', 'and',
        'vandeweghe', 'coco', 'after', 'belarus', 'defeated', 'americans',
        'the'],
       dtype='<U13')]

In [10]:
# now let's create a pandas API out of this
import pandas as pd

pd.options.display.max_columns = 999
df = pd.DataFrame(data=bag_words.toarray(),columns=count_vect.get_feature_names())

In [11]:
df # display the full bag of words matrix  

,12th,2000,2026,about,after,americans,and,archival,articles,at,be,belarus,best,bested,biggest,board,body,brazil,breeders,but,championships,claim,coco,commentary,companies,concern,court,cup,deciding,declaring,defeated,demanded,dispute,dominated,doubles,end,ended,event,events,experts,fifa,first,five,for,giant,governance,governing,has,hasn,he,horses,hosting,how,immediately,in,including,interest,inventor,is,issuing,it,its,japan,known,lindsey,marred,match,members,months,more,morocco,most,much,new,news,nonexecutive,of,old,one,patent,pays,popular,proposes,published,referees,retiring,rogers,said,scandal,shelby,shiffrin,since,slalom,soccer,some,sporting,spray,stage,stop,technical,than,the,their,thoroughbred,times,title,to,vandeweghe,vanishing,vonn,watched,who,will,with,won,world,year,york
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,2,1,0,0,1,1,1,0,0,0,0,0,0,1,0,0,0
2,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,2,0,0,0,0,1,0,0,0,1,0,0,0,0,2,0,0
3,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1
4,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1
5,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,2,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0
6,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
8,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
9,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0


In [12]:
# print out 10 most common words in our data
df.sum().sort_values()[-10:]

news          3
new           3
including     3
to            4
about         4
cup           4
and           6
world         6
in            7
the          17
dtype: int64

In [13]:
# print out 10 least common words in our data
df.sum().sort_values()[:10] # small sample size means most words occur one time

12th            1
referees        1
proposes        1
popular         1
pays            1
patent          1
one             1
old             1
nonexecutive    1
much            1
dtype: int64

# TF-IDF Conversion
We have a very small sample of data, but lets covert to tf-idf for the sake of programming it. Recall that Tf-idf transformation (default in `sklearn` is):

$$ \text{tf}(t,d) = f_{td}\text{, } t\in T \text{ and } d \in D $$

$$ \text{idf}(t,d) = \log{\frac{|D|}{|n_t|}}\text{, where } n_t=d\in D \text{ with } t\in d $$

$$\text{tf-idf}(t,d)=\text{tf}(t,d) \cdot (1+\text{idf}(t,d))$$

- http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer() # an object capable of counting words in a document!

tfidf_mat = tfidf_vect.fit_transform(summary_text) # that's it! its converted!!

In [15]:
# convert to pandas to get better idea about the data
df = pd.DataFrame(data=tfidf_mat.toarray(),columns=tfidf_vect.get_feature_names())
df

,12th,2000,2026,about,after,americans,and,archival,articles,at,be,belarus,best,bested,biggest,board,body,brazil,breeders,but,championships,claim,coco,commentary,companies,concern,court,cup,deciding,declaring,defeated,demanded,dispute,dominated,doubles,end,ended,event,events,experts,fifa,first,five,for,giant,governance,governing,has,hasn,he,horses,hosting,how,immediately,in,including,interest,inventor,is,issuing,it,its,japan,known,lindsey,marred,match,members,months,more,morocco,most,much,new,news,nonexecutive,of,old,one,patent,pays,popular,proposes,published,referees,retiring,rogers,said,scandal,shelby,shiffrin,since,slalom,soccer,some,sporting,spray,stage,stop,technical,than,the,their,thoroughbred,times,title,to,vandeweghe,vanishing,vonn,watched,who,will,with,won,world,year,york
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.229014,0.000000,0.000000,0.170324,0.229014,0.122941,0.000000,0.000000,0.000000,0.000000,0.229014,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00000,0.000000,0.00000,0.229014,0.229014,0.000000,0.000000,0.000000,0.0000,0.000000,0.229014,0.000000,0.229014,0.0000,0.0000,0.000000,0.229014,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.229014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.229014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.229014,0.000000,0.000000,0.229014,0.000000,0.229014,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.203324,0.229014,0.00000,0.000000,0.229014,0.151430,0.229014,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.229014,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.210963,0.139495,0.156900,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.139495,0.000000,0.210963,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.179338,0.000000,0.000000,0.0000,0.000000,0.210963,0.000000,0.000000,0.000000,0.000000,0.000000,0.210963,0.000000,0.000000,0.210963,0.210963,0.0000,0.102836,0.000000,0.210963,0.0000,0.000000,0.0000,0.210963,0.179338,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.210963,0.000000,0.210963,0.179338,0.210963,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.210963,0.000000,0.0000,0.000000,0.000000,0.210963,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.210963,0.0000,0.210963,0.0000,0.000000,0.000000,0.187298,0.000000,0.00000,0.000000,0.000000,0.139495,0.000000,0.0000,0.000000,0.210963,0.000000,0.000000,0.0000,0.000000,0.250549,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.241136,0.000000,0.000000,0.195770,0.271223,0.271223,0.000000,0.000000,0.000000,

In [16]:
# print out 10 words with max tfidf, normalized by document occurrence
df.max().sort_values()[-10:]

commentary       0.271223
including        0.271223
times            0.271223
thoroughbred     0.317370
championships    0.317370
breeders         0.317370
of               0.352283
the              0.367920
year             0.414406
slalom           0.424053
dtype: float64

# Working with (a bit) more data
What if we do not have the memory to deal with dense matrix representatioan and we need to keep it sparse?


In [17]:
from sklearn.datasets import fetch_20newsgroups
bunch = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)

the `bunch` object returned from sklearn is similar to a python dictionary. We can access different fields of the object with keys.

In [18]:
len(bunch.data)

11314

In [19]:
print(bunch.data[0]) # we should split this up by newlines

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [21]:
import numpy as np
#randomly grad an index
idx = round(np.random.rand()*len(bunch.data))
# and print it pretty
print("\n".join(bunch.data[idx].split("\n")))

From: system@garlic.sbs.com (Anthony S. Pelliccio)
Subject: Re: arcade style buttons and joysticks
Organization: Antone's Italian Kitchen and Excellence in Operating Network
X-Newsreader: rusnews v1.02
Lines: 26

dnewman@lynx.dac.northeastern.edu (David F. Newman) writes:

> Hi there,
> Can anyone tell me where it is possible to purchase controls found
> on most arcade style games.  Many projects I am working on would
> be greatly augmented if I could implement them.  Thanx in advance.
> 
> -Dave
> dnewman@lynx.dac.northeastern.edu
> 

Contact Chris Arthur at Chris_Arthur@pennies.stratus.com
He restores lots of old video and arcade games and knows where to get
parts.

Tony

-----------------------------------------------------------------------
-- Anthony S. Pelliccio, kd1nr/ae    // Yes, you read it right, the  //
-- system @ garlic.sbs.com          // man who went from No-Code    //
-----------------------------------// (Thhhppptt!) to Extra in     //
-- Flame Retardent Sysadmin     

In [22]:
%%time
news_tfidf = tfidf_vect.fit_transform(bunch.data) 

CPU times: user 3.92 s, sys: 125 ms, total: 4.04 s
Wall time: 4.1 s


In [23]:
news_tfidf.shape

(11314, 130107)

In [24]:
tfidf_vect.vocabulary_

{'hypocrisy': 64962,
 'kelleyb': 72176,
 'innoculate': 67402,
 '3ctp': 12658,
 'hzrck': 65053,
 'cscx': 43789,
 'r8bx82i': 98853,
 '8108': 20151,
 'interurban': 67963,
 'ddavis': 45454,
 'eventhandler': 52920,
 'shield': 106812,
 'cruelties': 43626,
 '23s': 9452,
 'microcomputer': 81853,
 'private_key': 95993,
 'jfox': 69960,
 'eeam': 50564,
 'marauders': 79408,
 '10460': 2714,
 'anatol': 28114,
 'kfri': 72412,
 'truths': 116716,
 '21864': 8890,
 'signatures': 107344,
 'o5xt': 88789,
 'gondor': 59759,
 'jtm': 70887,
 'mohrman': 83382,
 'ch41': 38878,
 'replication': 101025,
 'chod3horb_': 39474,
 'xlock': 126625,
 '1qiijs': 7487,
 'ranford': 99258,
 'commemorates': 41213,
 'offen': 89381,
 '_a6': 23473,
 'reconnected': 100089,
 '8vk': 21353,
 'utahn': 119867,
 'majesty': 79041,
 'hkj35': 63448,
 'cbr_128000': 38330,
 'outburst': 90783,
 'q8u': 97576,
 'diagrams': 47029,
 'instalation': 67560,
 '16702': 5312,
 'axles': 30872,
 '5650': 15482,
 '0wa8nfu': 2202,
 'additonal': 26080,
 '388e

In [25]:
# create pandas dataframe
vec = news_tfidf.max(axis=0)
df  = pd.DataFrame(data=vec.toarray(),columns=tfidf_vect.get_feature_names())

In [26]:
# largest tfidf 
df.max().sort_values()[-10:]

kk          0.870294
db          0.871473
scsi        0.875086
blah        0.879426
donoghue    0.891653
00          0.907726
___         0.908826
25          0.913127
forged      0.940511
ax          0.998314
dtype: float64

In [27]:
# now lets do the transformation with a smaller vocabulary
tfidf_vect = TfidfVectorizer(stop_words='english',
                             max_df=0.01,
                             min_df=4)
news_tfidf = tfidf_vect.fit_transform(bunch.data) 
print(news_tfidf.shape)
vec=news_tfidf.max(axis=0)
df = pd.DataFrame(data=vec.toarray(),columns=tfidf_vect.get_feature_names())
df.max().sort_values()[-10:]

(11314, 28592)


dialix            0.947270
blah              0.952881
ualberta          0.956563
stephanopoulos    0.959855
forged            0.971300
mufti             0.976947
ax                0.999881
meyers            1.000000
slower            1.000000
ucsd              1.000000
dtype: float64

# Using your own vocabulary

In [28]:
# read in scrabble dictionary from file
with open('data/ospd.txt') as f:
    vocab = f.read().split('\n')
    
# now lets do the transformation with a custom vocabulary
tfidf_vect = TfidfVectorizer(vocabulary=vocab)
news_tfidf = tfidf_vect.fit_transform(bunch.data) 
print(news_tfidf.shape)
vec=news_tfidf.max(axis=0)
df = pd.DataFrame(data=vec.toarray(),columns=tfidf_vect.get_feature_names())
df.max().sort_values()[-10:]

(11314, 79340)


incoming    0.925305
siemens     0.927114
water       0.928029
echo        0.947289
blah        0.951962
dos         0.953675
lib         0.954914
forged      0.978762
la          0.982737
ax          0.999999
dtype: float64

Looking for how to do a word cloud? Check this out:
- https://github.com/amueller/word_cloud

Want to perform more serious NLP with richer options:
- http://www.nltk.org